In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
from itertools import combinations, groupby
from collections import Counter
from pathlib import Path
from apriori_algo import AprioriAlgo
apriori = AprioriAlgo()

In [2]:
orders = pd.read_csv('../input/all_orders.csv', index_col = 0)
cluster_df = pd.read_csv('../input/cluster_df')
products = pd.read_csv('../input/products.csv')
aisles = pd.read_csv('../input/aisles.csv')
departments = pd.read_csv('../input/departments.csv')

c:\users\akshaysharma\hvacproakshay\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
orders = orders.sort_values(['order_id','product_id'])
orders

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,user_id,cluster
30264547,1,10246,3,0,83,4,fresh vegetables,produce,112108,1
30264545,1,11109,2,1,108,16,other creams cheeses,dairy eggs,112108,1
30264548,1,13176,6,0,24,4,fresh fruits,produce,112108,1
30264544,1,22035,8,1,21,16,packaged cheese,dairy eggs,112108,1
30264550,1,43633,5,1,95,15,canned meat seafood,canned goods,112108,1
...,...,...,...,...,...,...,...,...,...,...
23108244,3421083,21162,3,0,92,18,baby food formula,babies,25247,1
23108238,3421083,24852,9,1,24,4,fresh fruits,produce,25247,1
23108247,3421083,35211,5,0,92,18,baby food formula,babies,25247,1
23108242,3421083,39678,6,1,74,17,dish detergents,household,25247,1


In [4]:
all_orders = orders.set_index('order_id')['product_id'].rename('item_id')

In [7]:
all_orders.dtype

dtype('int64')

In [8]:
groupAll_association_rules = apriori.association_rules(all_orders,.0001)

Starting order_item:               33819106
Items with support >= 0.0001:           10949
Remaining order_item:              31116451
Remaining orders with 2+ items:     3136021
Remaining order_item:              30928016
Item pairs:                        21061284
Item pairs with support >= 0.0001:      61832



In [15]:
groupAll_association_rules.columns

Index(['item_A', 'item_B', 'freqAB', 'supportAB', 'freqA', 'supportA', 'freqB',
       'supportB', 'confidenceAtoB', 'confidenceBtoA', 'lift'],
      dtype='object')

In [16]:
groupAll_association_rules_copy = groupAll_association_rules.copy()

In [17]:
groupAll_association_rules_copy = groupAll_association_rules_copy.merge(products[['product_id','product_name']]
                   .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
groupAll_association_rules_copy = groupAll_association_rules_copy.merge(products[['product_id','product_name']]
           .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
groupAll_association_rules_copy = groupAll_association_rules_copy[['item_A','item_B','product_name_A',
                                                                'product_name_B','freqAB','supportAB','freqA',
                                                                'supportA','freqB','supportB','confidenceAtoB',
                                                                'confidenceBtoA','lift']]

In [18]:
groupAll_association_rules_copy.head()

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,41349,49570,Dairy Free Greek Yogurt Strawberry,Dairy Free Greek Yogurt Blueberry,317,0.000101,656,0.000209,602,0.000192,0.483232,0.526578,2517.316914
1,19244,26488,Grassfed Whole Milk Strawberry Yogurt,Organic Strawberry Grassfed Whole Milk Yogurt,326,0.000104,658,0.000210,731,0.000233,0.495441,0.445964,2125.461740
2,11224,39739,Organic Cashew Nondairy Blueberry Yogurt,Organic Nondairy Strawberry Cashew Yogurt,416,0.000133,687,0.000219,987,0.000315,0.605531,0.421479,1923.970475
3,15063,39739,Organic Cashew Nondairy Vanilla Yogurt,Organic Nondairy Strawberry Cashew Yogurt,331,0.000106,856,0.000273,987,0.000315,0.386682,0.335360,1228.615638
4,7657,19513,Hair Shampoos,Moroccan Argan Oil + Argan Stem Cell Triple Mo...,338,0.000108,714,0.000228,924,0.000295,0.473389,0.365801,1606.665542


In [19]:
groupAll_association_rules_copy.shape

(61832, 13)

In [20]:
print("Writing clusters aisle rules to csv...")
groupAll_association_rules_copy.to_csv('../input/associationRule_no_cluster/all_aisle_rules.csv')

Writing clusters aisle rules to csv...
